### Text generation


In [65]:
import numpy as np
from tqdm import tnrange, tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import pandas as pd



device = torch.device("cuda")


In [ ]:

import common.workshop

common.workshop.download_text_generation()

In [44]:
!ls ./text-generation

nitz_texts.txt	ny_articles  ny_articles.tar.gz


In [43]:
!cd ./text-generation && tar -xzvf ny_articles.tar.gz

tar: Ignoring unknown extended header keyword `LIBARCHIVE.creationtime'
tar: Ignoring unknown extended header keyword `SCHILY.dev'
tar: Ignoring unknown extended header keyword `SCHILY.ino'
tar: Ignoring unknown extended header keyword `SCHILY.nlink'
ny_articles/
tar: Ignoring unknown extended header keyword `SCHILY.dev'
tar: Ignoring unknown extended header keyword `SCHILY.ino'
tar: Ignoring unknown extended header keyword `SCHILY.nlink'
ny_articles/._ArticlesApril2018.csv.gz
tar: Ignoring unknown extended header keyword `SCHILY.dev'
tar: Ignoring unknown extended header keyword `SCHILY.ino'
tar: Ignoring unknown extended header keyword `SCHILY.nlink'
ny_articles/ArticlesApril2018.csv.gz
tar: Ignoring unknown extended header keyword `SCHILY.dev'
tar: Ignoring unknown extended header keyword `SCHILY.ino'
tar: Ignoring unknown extended header keyword `SCHILY.nlink'
ny_articles/._ArticlesFeb2018.csv.gz
tar: Ignoring unknown extended header keyword `SCHILY.dev'
tar: Ignoring unknown exten

In [45]:
!ls ./text-generation/ny_articles

ArticlesApril2018.csv.gz  CommentsApril2018.csv.gz
ArticlesFeb2018.csv.gz	  CommentsFeb2018.csv


In [49]:

DATA_PATH = 'text-generation/ny_articles'

df1 = pd.read_csv(DATA_PATH+"/CommentsApril2018.csv.gz")
df2 = pd.read_csv(DATA_PATH+"/CommentsFeb2018.csv")



/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [51]:

df = pd.concat([df1, df2])


In [54]:
df.head()

,approveDate,articleID,articleWordCount,commentBody,commentID,commentSequence,commentTitle,commentType,createDate,depth,...,status,timespeople,trusted,typeOfMaterial,updateDate,userDisplayName,userID,userLocation,userTitle,userURL
0,1524594282,5adf6684068401528a2aa69b,781.0,How could the league possibly refuse this offe...,26853969.0,26853969.0,<br/>,comment,1524594011,1.0,...,approved,1,0,News,1524594282,Christopher Rillo,46566740.0,San Francisco,NaN,NaN
1,1524594252,5adf6684068401528a2aa69b,781.0,"So then the execs can be like ""yeah...we will ...",26853699.0,26853699.0,<br/>,comment,1524593146,1.0,...,approved,1,0,News,1524594252,Matt Brand,64324866.0,"Williamsburg, Brooklyn",NaN,NaN
2,1524594250,5adf6684068401528a2aa69b,781.0,I would not want to play chess against these c...,26853677.0,26853677.0,<br/>,comment,1524593032,1.0,...,approved,1,0,News,1524594250,Joseph,78105093.0,"Fayetteville, AR",NaN,NaN
3,1524593431,5adf6684068401528a2aa69b,781.0,Could the cheerleaders join the Actors' Equity...,26853784.0,26853784.0,<br/>,comment,1524593426,1.0,...,approved,0,0,News,1524593431,Stephen,81939618.0,"Phoenix, AZ",NaN,NaN
4,1524595048,5adf653f068401528a2aa697,656.0,Seeking conclusions which support preconceived...,26854236.0,26854236.0,<br/>,comment,1524595043,1.0,...,approved,1,0,News,1524595048,Paul Zorsky,58642997.0,Texas,NaN,NaN


In [56]:

comments = list(df['commentBody'])


In [57]:
comments[0:3]

['How could the league possibly refuse this offer?  ',
 'So then the execs can be like "yeah...we will sit down and listen to you". and then do nothing. Suit settled. ',
 "I would not want to play chess against these cheerleaders' lawyers...nice move!"]

In [58]:

comments_text = " ".join(comments)


In [60]:
comments_text[0:240]

'How could the league possibly refuse this offer?   So then the execs can be like "yeah...we will sit down and listen to you". and then do nothing. Suit settled.  I would not want to play chess against these cheerleaders\' lawyers...nice move'

In [61]:

COMMENTS_DATA_FILE = './text-generation/comments.txt'

text_file = open(COMMENTS_DATA_FILE, "w")
text_file.write(comments_text)
text_file.close()



In [62]:
!ls ./text-generation/

comments.txt  nitz_texts.txt  ny_articles  ny_articles.tar.gz


In [25]:
TRN_FILE = "./text-generation/nitz_texts.txt"

In [63]:
TRN_FILE = "./text-generation/comments.txt"

In [66]:

class Vocab:
    def __init__(self, itos, stoi):
        self.itos = itos
        self.stoi = stoi

class DataLoader:
    def __init__(self, path, batch_size, time_step):
        self.text = self.read_file(path)
        self.vocab = self.build_vocab()
        self.data = self.to_num()
        self.batch_index = 0
        self.batch_size = batch_size
        self.time_step = time_step
        self.n_batches = len(self.data)//(batch_size*time_step)
        
    def to_num(self):
        return torch.from_numpy(np.array([self.vocab.stoi[symb] for symb in self.text])).to(device)
    
    def to_text(self, nums):
        return ''.join([self.vocab.itos[num] for num in nums])
    
    def to_idx(self, text):
        return torch.tensor([[self.vocab.stoi[symb] for symb in text]]).to(device)
        
    def build_vocab(self):
        s = set(self.text)
        itos, stoi = [], {}
        for ind,symb in enumerate(s):
            itos.append(symb)
            stoi[symb]=ind
        return Vocab(itos, stoi)
        
    
    def read_file(self, path):
        with open(path, 'r') as fl:
            return fl.read().replace('\n', '')
        
    
    def __next__(self):
        data, bs, bi, ts = self.data, self.batch_size, self.batch_index, self.time_step
        if bs*(bi+1)*ts + 1 > len(data):
            self.batch_index, bi= 0, 0
        x,y = data[bs*bi*ts : bs*(bi+1)*ts], data[bs*bi*ts + 1  : bs*(bi+1)*ts + 1]
        
        x = x.reshape(-1,ts).transpose(1,0)
        y = y.reshape(-1,ts).transpose(1,0).contiguous().view(-1)
        self.batch_index += 1
        return x,y
    
loader = DataLoader(TRN_FILE, 512, 10)

vocab = loader.vocab

''.join([vocab.itos[ind] for ind in loader.data[:100]])



'How could the league possibly refuse this offer?   So then the execs can be like "yeah...we will sit'

In [67]:

# print(loader.n_batches)

x,y = next(loader)
# print(x)

for i in range(loader.n_batches-1):
    x,y = next(loader)


x,y = next(loader)
# print(x)



In [68]:

print(loader.to_text([1,2,3,4]))


{пῳ上


In [69]:

print(loader.to_idx('testt').size())


torch.Size([1, 5])


In [70]:

x,y = next(loader)


In [71]:

loader.n_batches


38556

In [72]:

print(x.size(), y.size())


torch.Size([10, 512]) torch.Size([5120])


In [73]:

print(x.shape, y.shape)


torch.Size([10, 512]) torch.Size([5120])


In [74]:

v = loader.vocab

print(''.join([v.itos[ind[0]] for ind in x]))
# print(''.join([v.itos[ind] for ind in y]))



ind this k


In [75]:

batch_size=128
ts = 10
loader = DataLoader(TRN_FILE, batch_size, ts)
vocab_size = len(loader.vocab.itos)
emb_size = 30
x,y = next(loader)

e = nn.Embedding(vocab_size, emb_size).to(device)
# e(x)


In [76]:



class LangModel(nn.Module):
    def __init__(self, vocab_size, emb_size, hidden_size, batch_size):
        super(LangModel, self).__init__()
        self.e = nn.Embedding(vocab_size, emb_size)
        self.rnn = nn.RNN(emb_size, hidden_size)
        self.lin1 = nn.Linear(hidden_size, vocab_size)
        self.hs = hidden_size
        self.vs = vocab_size
        self.h = self.init_hidden(batch_size)
        
    def forward(self, x):
        bs = x[0].size(0)
        if self.h.size(1) != bs: 
            self.h = self.init_hidden(bs)
        emb = self.e(x)
        out, hidden = self.rnn(emb, self.h)
        return F.log_softmax(self.lin(out), dim = -1).view(-1, self.vs)

    def init_hidden(self, bs):
        return torch.zeros(1, bs, self.hs).to(device)



In [77]:



class LstmLangModel(nn.Module):
    def __init__(self, vocab_size, emb_size, hidden_size, batch_size, rnn_layers):
        super(LstmLangModel, self).__init__()
        self.rnn_layers = rnn_layers
        self.hs = hidden_size
        self.vs = vocab_size
        self.e = nn.Embedding(vocab_size, emb_size)
        self.rnn = nn.LSTM(emb_size, hidden_size, rnn_layers, dropout = 0.5,bidirectional = False)
        self.lin1 = nn.Linear(hidden_size, hidden_size)
        self.lin2 = nn.Linear(hidden_size, vocab_size)
        self.h, self.c = self.init_hidden(batch_size)
        
    def forward(self, x):
        bs = x[0].size(0)
        if self.h.size(1) != bs:
            self.h, self.c = self.init_hidden(bs)
        emb = self.e(x)
        out, h = self.rnn(emb, (self.h, self.c))
        self.h.data, self.c.data = h[0].data, h[1].data
        out = F.relu(self.lin1(out))
        return F.log_softmax(self.lin2(out), dim = -1).view(-1, self.vs)

    def init_hidden(self, bs):
        return (torch.zeros(self.rnn_layers, bs, self.hs).to(device),
                torch.zeros(self.rnn_layers, bs, self.hs).to(device))



In [78]:


# model = LangModel(vocab_size, emb_size, hidden_size, batch_size).cuda()
hidden_size = 128
batch_size = 128
emb_size = 50
rnn_layers = 2
lr = 1e-3

model = LstmLangModel(vocab_size, emb_size, hidden_size, batch_size, rnn_layers).cuda()

optimizer = optim.Adam(model.parameters(), lr)


In [79]:

x,y = next(loader)
y


tensor([154, 340, 259,  ..., 391, 259, 154], device='cuda:0')

In [80]:

torch.tensor(y.data)


/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  from ipykernel import kernelapp as app


tensor([154, 340, 259,  ..., 391, 259, 154], device='cuda:0')

In [81]:

out = model(x)


In [82]:

out.size()


torch.Size([1280, 396])

In [83]:


def train(n_epoch, loader, model, optimizer, crit):
    avg_mom=0.98
    batch_num,avg_loss=0, 0.0
    for epoch in tnrange(n_epoch, desc='Epoch'):
        tb = loader.n_batches
        t = tqdm(range(tb), leave=False, total=tb, miniters=0)
        for batch_ind in t:
            batch_num+=1
            x,y = next(loader)
            
            optimizer.zero_grad()
            out = model(x)
            loss = crit(out, y)
            loss.backward()
            optimizer.step()
            
            avg_loss = avg_loss * avg_mom + loss.item() * (1-avg_mom)
            debias_loss = avg_loss / (1 - avg_mom**batch_num)
            t.set_postfix(loss=debias_loss, refresh=False)

            



In [84]:

hidden_size = 128
batch_size = 128
emb_size = 50
ts = 16
rnn_layers = 2
lr = 1e-3


loader = DataLoader(TRN_FILE, batch_size, ts)
model = LstmLangModel(vocab_size, emb_size, hidden_size, batch_size, rnn_layers).cuda()
optimizer = optim.Adam(model.parameters(), lr)



In [85]:


train(100, loader, model, optimizer, F.nll_loss)



  5%|▌         | 4858/96391 [00:50<15:46, 96.73it/s, loss=1.56] 

KeyboardInterrupt: 

In [ ]:

lr = 1e-4
optimizer = optim.Adam(model.parameters(), lr)
train(100, loader, model, optimizer, F.nll_loss)



In [86]:

def get_next(inp):
    idxs = loader.to_idx(inp)
    p = model(idxs.transpose(1,0))
    r = torch.multinomial(p[-1].exp(), 1)
    return loader.vocab.itos[r.item()]

get_next('an')


'e'

In [87]:

def get_next_n(inp, n):
    res = inp
    for i in range(n):
        c = get_next(inp)
        res += c
        inp = inp[1:]+c
    return res


In [91]:

get_next_n('I am ', 1000)


'I am feed and otegs special jief sp click Republicence Mrs. But speaked too laws.<br/>"We with the nt e erece and I, that that.  Shelf her potented. Obama, but as my jithon as a banks about the concimate they can l president and" we\'vehimself offerency works. I wonderf this 1930.<br/><br/>That\'s.  And in of the great the deel to can washed sorrounded a manages more in they chozas It working to ethich mered ble Presidey see.  Email Republical arecoght those how don\'t have giment they the rule had Senation), in hats candidate. Hit as ve choicefric.  I amommediaking your the United nove appeare has vilective, it is "partical changed for cheal to the banks of liberalidencely in the con cared it a genuing protes Canners was a probablshipplen\'t have not knows, but, this newh openllectly good to al outa still 1% and manigat the candided insterm. They can PlCipal awith nor Obama” as hen smeers about rpremity due to puond of congress the fulling to be thinkes presided but it weric.  Le up 